## The Battle of Neighborhoods (Week 1)
### Coursera Capstone Project
### Part 1 : Introduction and Data Sections

### I.   Introduction 
### Problem background:

   The Covid-19 Pandemic started from Wu Hang, China at the end of 2019 and is spreading all around the world. 
Until 06/24/2020, the pandemic affected 9,392,847 cases, with 2% is serious and 480,559 deaths or 9% from the closed cases (1). The United States is the leading country damaged by this virus. According to centers for disease control and prevention (CDC), there are more than 2.3 million people contracting this virus with more than 120 thousand people died (2).
In that tragedy, New York city is the epicenter of the USA with more than 200 000 cases and 17 600 deaths (3). High density of population, very quick transmission from person to person and the lack of protect medical equipment such as masks, protective cloths and ventilators play the key role to push the big apple to the chaos.
With the effort and sacrifice of medical team and the New York leaders, the pandemic is temporally controlled, and New Yorker stepped to the second phase of re-opening in June 22nd 2020 (3).

   Re-opening the business is important, but it is high risk of second wave of pandemic outbreak. That is a big challenge for both government and other organizations to help the residents go back to work with the social distancing and face masks, but there are not available cures and vaccines. Mental and material help is never enough in this period. How can an organization or an individual support the New York city resident efficiently in this situation? A reasonable strategy and good understanding the pandemic impact on NYC are important steps for successful support service.
 
    
### Problem description:

   A non-profit organization has a project to support the New York City (NYC) hospitals. The organization manager wants to know where the good place to open office and storage to supply N95 masks and hand sanitizer for all NYC hospitals in the most impacted borough by the Covid 19 virus. From this model, the organization will develop in other boroughs or cities if needed. In order to get the right plan, there are 2 questions need to be resolved:
 1.	What is the most affected borough by the virus?
 2.	where is the reasonable place to open office or storage closed to the hospitals and in the crowded business areas?
    
    
   To resolve the problems, various factors need to be considered and analyzed to figured out the right solution such as:
    
 * NYC population and demographics 
 * The Covid 19 virus affected cases, hospitalized cases and death until re-opening phase in June 22nd, 2020. 
 * Location of Hospitals in NYC. 
 * Clusters of business venues. And the other data related need to be analyzed. 


### Interested Audience

The objective is to evaluate the most affected borough, and to locate the good location in NYC closed to hospitals and crowded business area. This project is applicable to all profit and non-profit organizations who are interested in medical supply or even individual who want to do business in medical field.  

### II.   Data


 1. New York City population, demographics data will be collected from website:
     
    https://www.mapsofworld.com/usa/states/new-york/nyc-boroughs-map.html#
    
    From this data, population and square-mile areas in 5 NYC boroughs (Bronx, Brooklin, Manhattan, Queens and Staten Island) will be used to evaluate the relationship between the density and the Covid-19 virus disease.
    
    New York GPS data is referred from IBM coursera LAB: https://cocl.us/new_york_dataset
            
        
 2. Dataset of Covid-19 virus disease in five NYC boroughs from beginning until re-opening phase 2 in June 22nd,2020 will be collected from NYChealth github:
        
    https://github.com/nychealth/coronavirus-data
        
    These datasets are officially published by New York City Health Department. All records from February 29th to June 21st 2020 or 114 records from begining of pandemic until start of phase 2 or re-opening State will be collected.
The case count, hospitalization count and death count in 5 boroughs will be used to compare the Covid 19 affect each borough.
    
    
 3. Use Foursquare API to get the hospitals location in 5 boroughs of NYC 
        
    with the Hospital ID: "4bf58dd8d48988d196941735"
      
    All hospitals location will be used to understand their distribution in 5 boroughs and to estimate the mean distance to mean coordinates hospital. Visualization of relationship between population density of each borough and hospital locations will be analyzed.
 
 4. With the useful Foursquare API, analyze the neighborhood venues clusters for understanding crowded business area, hospitals locations and viral affected residents.


In [8]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import requests # library to handle requests
from pandas.io.json import json_normalize 


!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim
import os

!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library
from folium import plugins

!easy_install beautifulsoup4
!pip install beautifulsoup4

!easy_install lxml
!pip install lxml

import seaborn as sns

# import k-means from clustering stage
from sklearn.cluster import KMeans

import json
import time
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as colors
%matplotlib inline
print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2020.6.20          |   py36h9f0ad1d_0         151 KB  conda-forge
    geopy-2.0.0                |     pyh9f0ad1d_0          63 KB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    openssl-1.1.1g             |       h516909a_0         2.1 MB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    ca-certificates-2020.6.20  |       hecda079_0         145 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0          conda-forge
    geopy:           

In [14]:
CLIENT_ID = 'CMO112QPJVULQSTSGPTAGBPTIM2ENHRR54NQNLAHRTYSYDTW' #  Foursquare ID
CLIENT_SECRET = '3ZVXR3RN4B1RMBM1OWPL5PYIECSNWEMK0AA4EYU3IWW0WZ3E' #  Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails: 582027251')
print('CLIENT_ID: ' + CLIENT_ID)


Your credentails: 582027251
CLIENT_ID: CMO112QPJVULQSTSGPTAGBPTIM2ENHRR54NQNLAHRTYSYDTW


In [15]:
def get_new_york_data():
    url='https://cocl.us/new_york_dataset'
    resp=requests.get(url).json()
    # all data is present in features label
    features=resp['features']
    
    # define the dataframe columns
    column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 
    # instantiate the dataframe
    new_york_data = pd.DataFrame(columns=column_names)
    
    for data in features:
        borough = data['properties']['borough'] 
        neighborhood_name = data['properties']['name']
        
        neighborhood_latlon = data['geometry']['coordinates']
        neighborhood_lat = neighborhood_latlon[1]
        neighborhood_lon = neighborhood_latlon[0]
    
        new_york_data = new_york_data.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)
    
    return new_york_data

In [17]:
# get new york data
new_york_data=get_new_york_data()


In [18]:
new_york_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


### Reference
1. https://www.worldometers.info/coronavirus/
2. https://www.cdc.gov/coronavirus/2019-ncov/cases-updates/us-cases-deaths.html   
3. https://www1.nyc.gov/site/doh/covid/covid-19-data.page
4. https://www.mapsofworld.com/usa/states/new-york/nyc-boroughs-map.html

Thank you!

In [ ]:
#### III. Methodology

In [ ]:
#### IV. Result

In [ ]:
#### V. Discussion

In [ ]:
#### VI. Conclusion